1 Remove small rects

In [90]:
import pdfplumber
import pandas as pd

selected_rects = []

def is_nested(rect, other_rect):
    #is_equal = other_rect['x0'] == rect['x0'] and other_rect['x1'] == rect['x1'] and other_rect['y0'] == rect['y0'] and other_rect['y1'] == rect['y1']
    contains = other_rect['x0'] <= rect['x0'] and other_rect['x1'] >= rect['x1'] and other_rect['y0'] <= rect['y0'] and other_rect['y1'] >= rect['y1']
    
    # print(f"Value of contains: {contains}")
    # print(f"other_rect['x0']: {other_rect['x0']}")
    # print(f"rect['x0']: {rect['x0']}")
    # print(f"other_rect['x1']: {other_rect['x1']}")
    # print(f"rect['x1']: {rect['x1']}")
    # print(f"other_rect['y0']: {other_rect['y0']}")
    # print(f"rect['y0']: {rect['y0']}")
    # print(f"other_rect['y1']: {other_rect['y1']}")
    # print(f"rect['y1']: {rect['y1']}")
    #return (not is_equal) and contains 
    return contains 
        # break #If other_rect contains rect, then rect is a nested rect and should be ignored

with pdfplumber.open("../mvfodevarer_side12.pdf") as pdf:
    print(pdf.pages)
    page = pdf.pages[0]
    #TODO: 
    # 1. Ignore rects with height = 0? (or under certain threshold)
    # 2. Ignore nested rects
    im = page.to_image(resolution = 400)

    # for i in range(0,len(page.rects)):
    for i in range(0,11):
    #Selected rects
        rect = page.rects[i]
        if rect['height'] < 1 or rect['width'] < 1: continue #1 Ignore small rectangles

        to_be_ignored = False

        for j in range(0,len(page.rects)):
            print(f"(i,j): ({i},{j})")
        # for j in range(0,10):
            other_rect = page.rects[j]
            if i == j:
                break
            elif is_nested(rect, other_rect):
                to_be_ignored = True
                break
            # elif j == len(page.rects)-1:
            #     selected_rects.append(rect)
            else:
                continue
        if not to_be_ignored: 
            print("added to selected rects")
            selected_rects.append(rect)
        
        print(f"Added rect with index {i}")
        # print(f"Rect index: {i}")
        # print(f"Rect height: {rect['height']}")
        # print(f"Rect width: {rect['width']}")
    im = im.draw_rects(selected_rects)
    im.show()

[<Page:1>]
(i,j): (0,0)
added to selected rects
Added rect with index 0
(i,j): (1,0)
Added rect with index 1
(i,j): (9,0)
(i,j): (9,1)
(i,j): (9,2)
(i,j): (9,3)
(i,j): (9,4)
(i,j): (9,5)
(i,j): (9,6)
(i,j): (9,7)
(i,j): (9,8)
(i,j): (9,9)
added to selected rects
Added rect with index 9
(i,j): (10,0)
(i,j): (10,1)
(i,j): (10,2)
(i,j): (10,3)
(i,j): (10,4)
(i,j): (10,5)
(i,j): (10,6)
(i,j): (10,7)
(i,j): (10,8)
(i,j): (10,9)
Added rect with index 10


In [ ]:
#x0,x1,y0,y1
(x0,x1,y0,y1) = (3,4,1,8) #Rect
(x0_o,x1_o,y0_o,y1_o) = (2,5,0,9) #Other rect - does it contain rect?

#LOGIC TO BE IMPLEMENTED
(not (x0_o == x0 and x1_o == x1 and y0_o == y0 and y1_o == y1)) and (x0_o <= x0 and x1_o >= x1 and y0_o <= y0 and y1_o >= y1)

True

In [62]:
#Drawing rects
selected_rects = []

with pdfplumber.open("../mvfodevarer_side12.pdf") as pdf:
    print(pdf.pages)
    page = pdf.pages[0]
    #TODO: 
    # 1. Ignore rects with height = 0? (or under certain threshold)
    # 2. Ignore nested rects
    im = page.to_image(resolution = 400)

    # for i in range(0,len(page.rects)):
    for i in range(0,20):
    #Selected rects
        rect = page.rects[i]
        # if rect['height'] < 1 or rect['width'] < 1: continue #1 Ignore small rectangles
        selected_rects.append(rect)
        # print(f"Rect index: {i}")
        # print(f"Rect height: {rect['height']}")
        # print(f"Rect width: {rect['width']}")
    im = im.draw_rects(selected_rects)
    im.show()

[<Page:1>]


2 Remove nested rects

In [52]:
selected_rects = []

with pdfplumber.open("../mvfodevarer_side12.pdf") as pdf:
    print(pdf.pages)
    page = pdf.pages[0]
    #TODO: 
    # 1. Ignore rects with height = 0? (or under certain threshold)
    # 2. Ignore nested rects
    im = page.to_image(resolution = 400)
    for i in range(0,20):
        #Selected rects
            rect = page.rects[i]
            selected_rects.append(rect)
            # print(f"Rect index: {i}")
            # print(f"Rect height: {rect['height']}")
            # print(f"Rect width: {rect['width']}")
            print(rect)
    im = im.draw_rects(selected_rects)
    im.show()

[<Page:1>]
{'x0': 51.83952, 'y0': 727.6416, 'x1': 545.27952, 'y1': 755.96112, 'width': 493.44000000000005, 'height': 28.31952000000001, 'pts': [(51.83952, 114.35839999999996), (545.27952, 114.35839999999996), (545.27952, 86.03887999999995), (51.83952, 86.03887999999995)], 'linewidth': 0, 'stroke': False, 'fill': True, 'evenodd': False, 'stroking_color': 0, 'non_stroking_color': (0.804688, 0.867188, 0.675781), 'mcid': None, 'tag': None, 'object_type': 'rect', 'page_number': 1, 'path': [('m', (51.83952, 114.35839999999996)), ('l', (545.27952, 114.35839999999996)), ('l', (545.27952, 86.03887999999995)), ('l', (51.83952, 86.03887999999995)), ('h',)], 'dash': None, 'top': 86.03887999999995, 'bottom': 114.35839999999996, 'doctop': 86.03887999999995}
{'x0': 56.63952, 'y0': 732.5616, 'x1': 540.35952, 'y1': 750.92112, 'width': 483.71999999999997, 'height': 18.359519999999975, 'pts': [(56.63952, 109.4384), (540.35952, 109.4384), (540.35952, 91.07888000000003), (56.63952, 91.07888000000003)], 'li

In [5]:
my_string = "mvfodevarer"
# my_pdf = my_string.pdf
my_pdf = pdfplumber.open("../mvfodevarer.pdf") #pdfplumber.pdf.PDF

my_pdf.pdf

AttributeError: 'PDF' object has no attribute 'pdf'